In [ ]:
!pip install haversine &> /dev/null
!pip install selenium &> /dev/null
!apt install chromium-chromedriver &> /dev/null

In [ ]:
import pandas as pd
import numpy as np
import selenium
import time
import haversine as hs
import warnings
import requests

from haversine import Unit
from geopy.geocoders import Nominatim
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

warnings.filterwarnings("ignore")

In [ ]:
#import all csv files needed
pubs_data = pd.read_csv("/content/data-wrangling-pubs-data.csv", index_col=[0])
police_data = pd.read_csv(
    "/content/data-wrangling-police_data.csv", index_col=[0]
)
metro_data = pd.read_csv("/content/raw-london-underground.csv", index_col=[0])
crime_data = pd.read_csv("/content/raw_london_sexual_crimes.csv", index_col=[0])

In [ ]:
pubs_data.head()

,name,postcode,latitude,longitude,popularity,price,rating
0,The Black Horse,EN5 4BW,51.653075,-0.206657,0.939551,1.0,8.1
1,Ye Olde Mitre Inne,EN5 5SJ,51.652979,-0.199367,0.982768,1.0,7.6
2,The Arkley,EN5 3EP,51.652533,-0.219573,0.987375,1.0,7.7
3,Railway Tavern,EN4 8RR,51.650059,-0.174692,0.950895,1.0,7.4
4,The Kings Head,EN5 5SN,51.653499,-0.200979,0.377777,1.0,6.6


In [ ]:
police_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude
0,4f75c889e4b0b24b0c1c4c98,Barnet Police Station,26-32 High St,Barnet,Barnet,EN5 5RU,Police Station,51.652283,-0.198046
1,886e605708d446fd6af7be02,Barnet Police Station,NaN,London,NaN,NaN,Police Station,51.654345,-0.201383
2,7687cbfe5d9743d0bcfd91b6,Metropolitan Police,111 High St,Barnet,NaN,EN5 5XY,Police Station,51.655584,-0.202649
3,dd8ab317b29242cec99126f6,Cat Hill,"Snt Base, 13 Cat Hill",Barnet,NaN,EN4 8HG,Police Station,51.643170,-0.162268
4,fa389098b2a249fb9381e3f3,Metropolitan Police Service,13 Cat Hill,Barnet,NaN,EN4 8HG,Police Station,51.643139,-0.161936


In [ ]:
metro_data.head()

,name,latitude,longitude,line
0,Acton Town,51.502500,-0.278126,"District, Piccadilly"
1,Acton Central,51.508835,-0.263033,London Overground
2,Acton Central,51.508560,-0.262880,London Overground
3,Aldgate,51.513940,-0.075370,Metropolitan
4,Aldgate East,51.515140,-0.071780,"District, Hammersmith & City"


In [ ]:
crime_data.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
1,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.109640,51.520624,On or near Leather Lane,E01000916,Camden 027B,Violence and sexual offences,Under investigation,NaN
11,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.097562,51.518864,On or near Parking Area,E01000001,City of London 001A,Violence and sexual offences,Unable to prosecute suspect,NaN
21,fcd78bb549a08c5f2cbfbb0c70b6ba8e5dd61fa8ebe1b9...,2021-12,City of London Police,City of London Police,-0.095026,51.518499,On or near Park/Open Space,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
22,f4afa08625bdc8161e9dc0b14b39049581e5bc3a643e6f...,2021-12,City of London Police,City of London Police,-0.092628,51.517920,On or near St Alphage Garden,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
23,419176d79450dc395d689eed75c05ec59b6e84b7d43159...,2021-12,City of London Police,City of London Police,-0.092723,51.517032,On or near Aldermanbury Square,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN


### Get number of crimes in a radius of 500m of each pub



In [ ]:
crime_data.rename(
    {
        "Crime ID": "crime_id",
        "Month": "month",
        "Reported by": "reported_by",
        "Falls within": "falls_within",
        "Longitude": "longitude",
        "Latitude": "latitude",
        "Location": "location",
        "Crime type": "crime_type",
    },
    axis=1,
    inplace=True,
)

In [ ]:
def get_number_of_crimes(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    crimes = 0
    for latitude2, longitude2 in list(zip(df["latitude"], df["longitude"])):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance <= 500:
            crimes += 1
    return crimes

In [ ]:
pubs_data["number_of_crimes"] = pubs_data.apply(
    lambda row: get_number_of_crimes(row["latitude"], row["longitude"], crime_data),
    axis=1,
)

In [ ]:
pubs_data.head()

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes
0,The Black Horse,EN5 4BW,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30
1,Ye Olde Mitre Inne,EN5 5SJ,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45
2,The Arkley,EN5 3EP,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2
3,Railway Tavern,EN4 8RR,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64
4,The Kings Head,EN5 5SN,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40


### Get distance between pubs and metro, police station

In [ ]:
#calculate the distance between two venues and get the smallest distance
def get_distance(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    distance_min = 1000000
    for latitude2, longitude2 in list(zip(df.latitude, df.longitude)):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance < distance_min:
            distance_min = distance
    return distance_min

In [ ]:
#get distance between a pub and the nearest metro station
pubs_data["distance_metro"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], metro_data),
    axis=1,
)

In [ ]:
#get distance between a pub and the nearest police station
pubs_data["distance_police"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], police_data), axis=1
)

In [ ]:
# check the pubs with longest distance of a metro station
pubs_data.sort_values("distance_metro", ascending=False).head(10)

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police
74,The Royal Oak,BR6 6BJ,51.351319,0.089519,0.904898,1.0,7.0,16421.666549,245.869158
78,The Queens Head,BR6 6BQ,51.352253,0.089814,0.988041,1.0,6.2,16343.800356,140.243101
81,The Buff,BR6 9NL,51.360007,0.087786,0.935304,1.0,5.7,15539.261192,739.373431
73,The Woodman,BR6 7BA,51.359255,0.067621,0.938068,1.0,6.6,14900.304645,1682.199072
85,The Harvest Moon,BR6 0LQ,51.377816,0.101193,0.957069,1.0,5.0,14499.138570,86.023654
75,Ye Olde Whyte Lyon,BR6 8NE,51.364223,0.056685,0.909086,1.0,6.4,14060.927978,2603.929158
71,The British Queen,BR6 8NL,51.366437,0.054200,0.946402,1.0,7.3,13763.706988,2873.362006
69,Black Horse,BR6 8NW,51.368235,0.056846,0.954741,1.0,7.5,13668.240422,2826.749082
77,The Fox Inn,BR2 6BQ,51.361966,0.027390,0.938291,1.0,6.4,13509.864815,4450.039902
72,The Greyhound,BR2 6BP,51.363645,0.028593,0.956841,1.0,7.4,13364.667697,4412.322506


In [ ]:
#check the pubs with longest distance of a police station
pubs_data.sort_values("distance_police", ascending=False).head(10)

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police
77,The Fox Inn,BR2 6BQ,51.361966,0.027390,0.938291,1.0,6.4,13509.864815,4450.039902
72,The Greyhound,BR2 6BP,51.363645,0.028593,0.956841,1.0,7.4,13364.667697,4412.322506
76,The Chequers,BR2 8AP,51.390280,0.045439,0.975612,1.0,6.5,11135.478054,4109.081646
696,The Tichenham Inn,UB10 8DF,51.564438,-0.444744,0.959818,1.0,5.4,362.630888,3032.167296
638,The Moon & Sixpence,HA5 4HS,51.608274,-0.373095,0.979877,1.0,6.1,1806.176667,2997.865839
891,The Crown,SE12 0AJ,51.443711,0.017690,0.977309,1.0,6.3,5173.759942,2922.538201
707,The Plough Inn,UB2 4LG,51.494919,-0.365912,0.984589,1.0,6.6,1771.098095,2879.429542
71,The British Queen,BR6 8NL,51.366437,0.054200,0.946402,1.0,7.3,13763.706988,2873.362006
69,Black Horse,BR6 8NW,51.368235,0.056846,0.954741,1.0,7.5,13668.240422,2826.749082
75,Ye Olde Whyte Lyon,BR6 8NE,51.364223,0.056685,0.909086,1.0,6.4,14060.927978,2603.929158


### Get number of metro stations and police stations in a radius of 500m

In [ ]:
def request_police_foursquare(latitude, longitude):
    try:
        coordinate = f"{latitude},{longitude}"
        token = config("TOKEN")
        headers = {"Accept": "application/json", "Authorization": f"{token}"}
        url = f"https://api.foursquare.com/v3/places/search?ll={coordinate}&radius=500&categories=12072&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50"
        response = requests.request("GET", url, headers=headers)
        json_file = response.json()
        print(json_file)
        count = len(json_file["results"])
        return count
    except:
        print(coordinate)
        print(response)
        count = np.nan
        return count


In [ ]:
def request_metro_google(latitude, longitude):
    try:
        coordinate = f"{latitude},{longitude}"
        secret_key = config("SECRET_KEY")
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={coordinate}&radius=500&type=subway_station&key={secret_key}"
        payload = {}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)
        print(response.text)
        json_file = response.json()
        count = len(json_file["results"])
        return count
    except:
        print(coordinate)
        print(response)
        count = np.nan
        return count

In [ ]:
pubs_data["number_of_metro_stations"] = pubs_data.apply(lambda row: request_metro_google(row["latitude"], row["longitude"]), axis=1)

Streaming output truncated to the last 5000 lines.

51.49595,-0.099705
{
   "html_attributions" : [],
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 51.4946011,
               "lng" : -0.100329
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 51.4964629802915,
                  "lng" : -0.09931606970849795
               },
               "southwest" : {
                  "lat" : 51.4937650197085,
                  "lng" : -0.102014030291502
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png",
         "icon_background_color" : "#7B9EB0",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet",
         "name" : "Elephant & Castle",
         "photos" : [
            {
               "height" : 3024,
    

In [ ]:
pubs_data["number_of_police_stations"] = pubs_data.apply(lambda row: request_police_foursquare(row["latitude"], row["longitude"]), axis=1)

{'results': [{'fsq_id': '886e605708d446fd6af7be02', 'categories': [{'id': 12072, 'name': 'Police Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_policestation_', 'suffix': '.png'}}], 'geocodes': {'main': {'latitude': 51.654345, 'longitude': -0.201383}, 'roof': {'latitude': 51.654345, 'longitude': -0.201383}}, 'location': {'admin_region': 'England', 'country': 'GB', 'formatted_address': 'London', 'locality': 'London', 'post_town': 'Barnet'}, 'name': 'Barnet Police Station'}, {'fsq_id': '7687cbfe5d9743d0bcfd91b6', 'categories': [{'id': 12072, 'name': 'Police Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_policestation_', 'suffix': '.png'}}], 'geocodes': {'main': {'latitude': 51.655584, 'longitude': -0.202649}, 'roof': {'latitude': 51.655584, 'longitude': -0.202649}}, 'location': {'address': '111 High St', 'admin_region': 'England', 'country': 'GB', 'formatted_address': '111 High St, Barnet, EN5 5XY', 'loca

In [ ]:
pubs_data[pubs_data["number_of_police_stations"].isnull()]

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,number_of_metro_stations,number_of_police_stations
950,The Abbey Tap,E15 4QS,51.540283,-0.000077,0.974341,1.0,6.3,203.69449,302.190783,302,0,NaN


In [ ]:
token = config("TOKEN")
coordinate = f"51.540283,-0.000077"
headers = {"Accept": "application/json", "Authorization": f"{token}"}
url = f"https://api.foursquare.com/v3/places/search?ll={coordinate}&radius=500&categories=12072&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50"
response = requests.request("GET", url, headers=headers)
json_file = response.json()
print(json_file)
count = len(json_file["results"])

{'results': [{'fsq_id': '50895609e4b0b52a6b548d7a', 'categories': [{'id': 12072, 'name': 'Police Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_policestation_', 'suffix': '.png'}}], 'geocodes': {'main': {'latitude': 51.53917, 'longitude': 0.004005}, 'roof': {'latitude': 51.53917, 'longitude': 0.004005}}, 'location': {'address': '18 West Ham Lane', 'admin_region': 'England', 'country': 'GB', 'cross_street': '', 'formatted_address': '18 West Ham Lane, Stratford, Greater London, E15 4SG', 'locality': 'Stratford', 'neighborhood': ['Stratford'], 'post_town': 'London', 'postcode': 'E15 4SG', 'region': 'Greater London'}, 'name': 'Stratford Police Station'}, {'fsq_id': '9600f4829d3c4a7cfc80adf4', 'categories': [{'id': 12072, 'name': 'Police Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_policestation_', 'suffix': '.png'}}], 'geocodes': {'main': {'latitude': 51.539221, 'longitude': 0.003945}, 'roof': {'latitude'

In [ ]:
pubs_data["number_of_police_stations"][pubs_data["number_of_police_stations"].isnull()] = count

In [ ]:
pubs_data.loc[950]

name                         The Abbey Tap
postcode                           E15 4QS
latitude                         51.540283
longitude                        -0.000077
popularity                        0.974341
price                                  1.0
rating                                 6.3
distance_metro                   203.69449
distance_police                 302.190783
number_of_crimes                       302
number_of_metro_stations                 0
number_of_police_stations              4.0
Name: 950, dtype: object

In [ ]:
pubs_data.rename(columns={"number_of_metro_stations": "number_of_metro", "number_of_police_stations": "number_of_police"}, inplace=True)

In [ ]:
pubs_data.head()

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,number_of_metro,number_of_police
0,The Black Horse,EN5 4BW,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30,0,2.0
1,Ye Olde Mitre Inne,EN5 5SJ,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45,1,3.0
2,The Arkley,EN5 3EP,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2,0,0.0
3,Railway Tavern,EN4 8RR,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64,0,0.0
4,The Kings Head,EN5 5SN,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40,0,3.0


### Get borough

In [ ]:
postcode_list = pubs_data["postcode"].tolist()

In [ ]:
browser = webdriver.Chrome(options=options)

In [ ]:
borough_list = []
for postcode in postcode_list:
  try:
    browser.get("https://www.gov.uk/find-local-council")
    browser.find_element_by_id("postcode").clear()
    browser.find_element_by_id("postcode").send_keys(postcode)
    browser.find_element_by_css_selector(".gem-c-button.gem-c-button--bottom-margin").click()
    time.sleep(0.5)
    borough = browser.find_element_by_css_selector(".local-authority-results p.govuk-body:first-child").text
    #print(borough)
    borough_list.append(borough)
  except:
    print(postcode)
    borough_list.append(np.nan)

HA20 2HA
SW6
 SW7
 Sw3


In [ ]:
len(borough_list)

865

In [ ]:
pubs_data.shape

(865, 12)

In [ ]:
pubs_data["borough"] = borough_list

In [ ]:
pubs_data[pubs_data["borough"].isnull()]

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,number_of_metro,number_of_police,borough
47,The Fusilier,HA20 2HA,51.561739,-0.331929,0.960519,1.0,7.2,595.956601,2138.848420,30,0,0.0,NaN
551,McGettigan's Fulham,SW6,51.479888,-0.196053,0.955144,1.0,6.7,94.942344,273.991747,164,1,1.0,NaN
571,Gloucester Arms,SW7,51.498752,-0.183882,0.960377,1.0,5.9,612.484689,978.313212,108,1,0.0,NaN
760,The Sydney Arms,Sw3,51.488642,-0.169781,0.946835,1.0,6.8,638.109791,437.177332,111,0,2.0,NaN


In [ ]:
pubs_data["borough"][pubs_data["postcode"] == "HA20 2HA"] = "London Borough of Brent"
pubs_data["postcode"][pubs_data["postcode"] == "HA20 2HA"] = "HA0 2HA"

In [ ]:
pubs_data["borough"][pubs_data["postcode"] == "SW6"] = "London Borough of Hammersmith & Fulham"
pubs_data["postcode"][pubs_data["postcode"] == "SW6"] = "SW6 1AA"

In [ ]:
pubs_data["borough"][pubs_data["postcode"] == " SW7"] = "Royal Borough of Kensington and Chelsea"
pubs_data["postcode"][pubs_data["postcode"] == " SW7"] = "SW7 4RB"

In [ ]:
pubs_data["borough"][pubs_data["postcode"] == " Sw3"] = "Royal Borough of Kensington and Chelsea"
pubs_data["postcode"][pubs_data["postcode"] == " Sw3"] = "SW3 6NJ"

In [ ]:
columns_order = ["name", "postcode", "borough", "latitude", "longitude", "price", "rating", "popularity", "number_of_crimes", "number_of_metro", "number_of_police", "distance_metro", "distance_police"]
pubs_data = pubs_data.reindex(columns=columns_order)

In [ ]:
pubs_data.head()

,name,postcode,borough,latitude,longitude,price,rating,popularity,number_of_crimes,number_of_metro,number_of_police,distance_metro,distance_police
0,The Black Horse,EN5 4BW,London Borough of Barnet,51.653075,-0.206657,1.0,8.1,0.939551,30,0,2.0,904.722178,390.281423
1,Ye Olde Mitre Inne,EN5 5SJ,London Borough of Barnet,51.652979,-0.199367,1.0,7.6,0.982768,45,1,3.0,443.945965,119.561106
2,The Arkley,EN5 3EP,London Borough of Barnet,51.652533,-0.219573,1.0,7.7,0.987375,2,0,0.0,1767.173867,1215.815963
3,Railway Tavern,EN4 8RR,London Borough of Barnet,51.650059,-0.174692,1.0,7.4,0.950895,64,0,0.0,1343.248049,1149.621255
4,The Kings Head,EN5 5SN,London Borough of Barnet,51.653499,-0.200979,1.0,6.6,0.377777,40,0,3.0,568.292759,98.112834


In [ ]:
pubs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              865 non-null    object 
 1   postcode          865 non-null    object 
 2   borough           865 non-null    object 
 3   latitude          865 non-null    float64
 4   longitude         865 non-null    float64
 5   price             865 non-null    float64
 6   rating            865 non-null    float64
 7   popularity        865 non-null    float64
 8   number_of_crimes  865 non-null    int64  
 9   number_of_metro   865 non-null    int64  
 10  number_of_police  865 non-null    float64
 11  distance_metro    865 non-null    float64
 12  distance_police   865 non-null    float64
dtypes: float64(8), int64(2), object(3)
memory usage: 88.0+ KB


In [71]:
pubs_data["borough"].value_counts()

City of Westminster                        80
London Borough of Camden                   67
London Borough of Wandsworth               55
London Borough of Hackney                  51
London Borough of Richmond upon Thames     47
London Borough of Tower Hamlets            44
London Borough of Hammersmith & Fulham     41
Royal Borough of Kensington and Chelsea    40
London Borough of Islington                39
London Borough of Southwark                30
London Borough of Lewisham                 29
Royal Borough of Kingston upon Thames      29
Royal Borough of Greenwich                 27
London Borough of Haringey                 25
London Borough of Ealing                   24
London Borough of Waltham Forest           22
London Borough of Merton                   21
London Borough of Enfield                  21
London Borough of Newham                   20
London Borough of Hounslow                 18
London Borough of Croydon                  16
London Borough of Sutton          

In [72]:
pubs_data.drop(pubs_data[pubs_data["borough"] == "Surrey County Council"].index, inplace=True)

In [73]:
pubs_data.drop(pubs_data[pubs_data["borough"] == "Hertfordshire County Council"].index, inplace=True)

In [74]:
pubs_data["borough"].value_counts()

City of Westminster                        80
London Borough of Camden                   67
London Borough of Wandsworth               55
London Borough of Hackney                  51
London Borough of Richmond upon Thames     47
London Borough of Tower Hamlets            44
London Borough of Hammersmith & Fulham     41
Royal Borough of Kensington and Chelsea    40
London Borough of Islington                39
London Borough of Southwark                30
Royal Borough of Kingston upon Thames      29
London Borough of Lewisham                 29
Royal Borough of Greenwich                 27
London Borough of Haringey                 25
London Borough of Ealing                   24
London Borough of Waltham Forest           22
London Borough of Merton                   21
London Borough of Enfield                  21
London Borough of Newham                   20
London Borough of Hounslow                 18
London Borough of Croydon                  16
London Borough of Sutton          

In [77]:
pubs_data.reset_index(drop=True, inplace=True)

In [78]:
pubs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              859 non-null    object 
 1   postcode          859 non-null    object 
 2   borough           859 non-null    object 
 3   latitude          859 non-null    float64
 4   longitude         859 non-null    float64
 5   price             859 non-null    float64
 6   rating            859 non-null    float64
 7   popularity        859 non-null    float64
 8   number_of_crimes  859 non-null    int64  
 9   number_of_metro   859 non-null    int64  
 10  number_of_police  859 non-null    float64
 11  distance_metro    859 non-null    float64
 12  distance_police   859 non-null    float64
dtypes: float64(8), int64(2), object(3)
memory usage: 87.4+ KB


In [79]:
pubs_data.to_csv("data-tidying-pubs-data.csv")